In [ ]:
# Uncomment the below command if you don't have openpyxl
#!pip install openpyxl 

In [1]:
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")

In [3]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = [48]
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['period'] = pd.Series(period)

market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])
market_data['date'] = market_data['Time (UTC+10)'].dt.date
market_data.columns

Index(['Time (UTC+10)', 'Regions NSW Trading Price ($/MWh)',
       'Regions SA Trading Price ($/MWh)', 'Regions TAS Trading Price ($/MWh)',
       'Regions VIC Trading Price ($/MWh)',
       'Regions NSW Trading Total Intermittent Generation (MW)',
       'Regions SA Trading Total Intermittent Generation (MW)',
       'Regions TAS Trading Total Intermittent Generation (MW)',
       'Regions VIC Trading Total Intermittent Generation (MW)',
       'Regions NSW Operational Demand (MW)',
       'Regions SA Operational Demand (MW)',
       'Regions TAS Operational Demand (MW)',
       'Regions VIC Operational Demand (MW)', 'period', 'date'],
      dtype='object')

### Battery Class

In [247]:
###################################
# A Class to store battery functionaly such as revenue, charge and discharge
# periods, charge and discharge spot prices, charge and discharge market dispatch
#
# Consists of:
#      - charge_period : period when it should charge.
#      - discharge_period : period when it should discharge.
#      - charge_price : spot price given charging period.
#      - discharge_price : spot price given discharging period.
#      - charge_market_dispatch : set amount of market dispatch given charging period.
#      - discharge_market_dispatch : set amount of market dispatch given discharging period.
# 
# Functions:
#      - ComputeRevenue: To calculate revenue given discharge and charge period pairs
#
# Created by: Gilbert
###################################
class Battery:
    def __init__(self, charge_period, charge_spot_price,
                 discharge_period, discharge_spot_price):
        
        self.mlf = 0.991 # Set marginal loss factor in the specs.
        self.battery_capacity = 580
        self.battery_power = 300
        self.charge_efficiency = 0.9
        self.discharge_efficiency = 0.9
        
        self.charge_period = charge_period
        self.discharge_period = discharge_period
        
        self.charge_price = charge_spot_price
        self.discharge_price = discharge_spot_price
        
                
    def Revenue(self):
        if self.charge_market_dispatch == [] or self.discharge_market_dispatch == []:
            return 0
        
        # Spot Prices
        charge_sp = np.array(self.charge_price)[:, 1]
        discharge_sp = np.array(self.discharge_price)[:, 1]
        
        # Market Dispatches
        charge_md = np.array(self.charge_market_dispatch).T
        discharge_md = np.array(self.discharge_market_dispatch).T
        
        # Revenues
        charge_revenue = (charge_sp @ charge_md) * (1 / self.mlf)
        discharge_revenue = (discharge_sp @ discharge_md) * (self.mlf)
        
        return discharge_revenue + charge_revenue
    
    def NormalSetting(self):
        
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
        
        self.charge_capacity = [[0, 0] for i in range(len_charge)]
        self.discharge_capacity = [[0, 0] for i in range(len_discharge)]
        
        # CHARGE PERIOD --------------------------------------------------------------------
        for t in range(len_charge):
            # RAW_POWER = -MIN(BATTERY_POWER, (BATTERY_CAPACITY - OPENING_CAPACITY[t]) / CHARGE_EFFICIENCY * 2)
            self.charge_raw_power[t] = -min(battery_power, 
                                           (battery_cap - self.charge_capacity[t][OPENING]) / 
                                            self.charge_efficiency * 2)

            # MARKET_DISPATCH[t] = RAW_POWER / 2
            self.charge_market_dispatch[t] = self.charge_raw_power[t] / 2

            # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
            #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
            self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                        self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                        battery_cap))

            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_charge:
                self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------

        # Set DISCHARGE CAPACITY AT t = 0 to be the the last t of CHARGING CAPACITY
        self.discharge_capacity[0][OPENING] = self.charge_capacity[-1][CLOSING]

        for t in range(len_discharge):
            # RAW_POWER = MIN(BATTERY_POWER, OPENING_CAPACITY[t] / DISCHARGE EFFICIENCY * 2)
            self.discharge_raw_power[t] = min(battery_power,
                                              self.discharge_capacity[t][OPENING] /
                                              self.discharge_efficiency * 2)

            # MARKET_DISPATCH[t] = RAW_POWER[t] / 2 * DISCHARGE EFFICIENCY
            self.discharge_market_dispatch[t] = self.discharge_raw_power[t] / 2 * self.discharge_efficiency

            # CLOSING CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] -
            #                        MARKET_DISPATCH[t] * (1/DISCHARGE_EFFICIENCY), BATTERY_CAPACITY))
            self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] -
                                                            self.discharge_market_dispatch[t] * (1 / self.discharge_efficiency),
                                                            battery_cap))
            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_discharge:
                self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
     
        
    def Optimise(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        if (len_charge >= MAX_CHARGE_PERIOD and len_discharge >= MAX_DISCHARGE_PERIOD):
            return
        elif (len_charge == len_discharge):
            return
        
        if (len_charge - len_discharge == 1):
            self.discharge_capacity[-1][CLOSING] = 0
            self.discharge_capacity[-1][OPENING] = self.discharge_market_dispatch[-1]
            for t in range(1, len_discharge):
                self.discharge_capacity[-t - 1][CLOSING] = self.discharge_capacity[-t][OPENING]
                self.discharge_capacity[-t - 1][OPENING] = self.discharge_capacity[-t - 1][CLOSING] + self.discharge_market_dispatch[-t - 1] / self.discharge_efficiency
        
            self.charge_capacity[-1][CLOSING] = self.discharge_capacity[0][OPENING]
            self.charge_market_dispatch[-1] = -(self.charge_capacity[-1][CLOSING] - self.charge_capacity[-1][OPENING]) / self.charge_efficiency
            self.charge_raw_power[-1] = self.charge_market_dispatch[-1] * 2
        

### Helper function for battery optimisation

In [248]:
###################################
# A function to get spot prices based on selected regions.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - selected_periods : selected period for charging or discharging.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#
# Return:
#      - List of spot prices given period
#
# Created by: Gilbert
###################################
def GetSpotPrice(data, selected_periods, region = 'VIC'):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    spot_price = np.array(spot_price)
    
    # Find the spot prices from selected region. Periods are
    # index + 1, therefore to use the index we need to subtract
    # it by 1.
    retrieved_prices = []
    for period in selected_periods:
        # append(minimum or maximum ranking, spot_price[index])
        retrieved_prices.append((period[1], spot_price[period[1] - 1]))
    return retrieved_prices

###################################
# A function to find minimum and maximum point rank given threshold.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#      - buy_threshold : maximum number of buying point, default has been set to optimise Checkpoint 3.
#      - sell_threshold : maximum number of selling point, default has been set to optimise Checkpoint 3.
#
# Return:
#      - List of selected minimum point, list of selected maximum point
#
# Created by: Gilbert
###################################
def GetMinMax(data, region = 'VIC', buy_threshold = 13, sell_threshold = 4):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
        
    price = np.array(spot_price)
    minimum_price = np.argsort(price, kind = 'merge*sort') # (O(NlogN)), mergesort the minimum prices.
    maximum_price = minimum_price[::-1][:48] # (O(N)), maximum is the reverse order of minimum.
    
    selected_min_price = ['' for i in minimum_price] # (O(N)), set an empty array for the whole period.
    selected_max_price = ['' for i in minimum_price] # (O(N)), set an empty array for the whole period.
    
    # Select the lowest price spot over the given
    # buy_threshold as the minimum buying point.
    i = 0
    for b_t in range(buy_threshold):
        selected_min_price[minimum_price[i]] = b_t + 1
        i += 1
        
    # Select the highest price spot over the given
    # sell_threshold as the maximum selling point.
    i = 0
    for s_t in range(sell_threshold):
        selected_max_price[maximum_price[i]] = s_t + 1
        i += 1
        
    return selected_min_price, selected_max_price

###################################
# A function to Find Battery Charge and Discharge pairs in backward order.
# Backward order from 48th period to the 1st.
#
# Parameters:
#      - buy_period : Selected minimum price point as it will be where we buy energy for charging.
#      - sell_period : Selected maximum price point as it will be where we sell energy for discharging.
#
# Return:
#      - List of battery class pairs
#
# Created by: Gilbert
###################################
def FindBatteryPairs(buy_period, sell_period):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR

    period = 48
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    # Iterate over the whole period backwards
    for p in range(period - 1, -1, -1):
        # If maximum selling point is not empty, add (order, period)
        # as key-value pair into the OrderedDict.
        if sell_period[p] != '':
            # If battery buying point period is less MAXIMUM SELLING 
            # PERIOD PER PAIR, add new period.
            if len(sell) < MAX_SELL_PERIOD:
                sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
            # else, if battery selling point is full and there is 
            # higher maximum selling point then remove the lowest
            # selling point and add the new one into Dictionary.    
            else:
                max_key = max(sell, key=sell.get)
                if sell_period[p] < max_key:
                    sell.pop(max_key)
                    sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1

        # If battery selling point is not empty and minimum buying 
        # point is not empty.
        if len(sell) != 0 and buy_period[p] != '':
            # If battery buying point period is less MAXIMUM BUYING 
            # PERIOD PER PAIR, add new period.
            if len(buy) < MAX_BUY_PERIOD and len(buy) < math.ceil(len(sell) * 1.25):
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            # else, if battery buying point is full and there is 
            # lower minimum buying point then remove the highest
            # buying point and add the new one into Dictionary.
            else:
                max_key = max(buy, key=buy.get)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        # If the next period is not empty and battery buying point
        # is not empty then battery charge-discharge pair has been
        # created.
        # Reinitialise a new battery setup.
        if sell_period[p - 1] != '' and len(buy) != 0:
            battery.append([list(sell.items()), list(buy.items())])
            sell = OrderedDict()
            buy = OrderedDict()
    # Add the last battery charge-discharge pair occuring 
    # before 1st period.
    battery.append([list(sell.items()), list(buy.items())]) 

    # Check whether there is too many selling points, then
    # remove selling point until the number of selling points
    # is equal to the number of buying points while removing
    # the lowest selling point.
    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list
        
    return battery

###################################
# A function to set optimal charge and discharge amount of battery pairs.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - battery_pairs : list of all battery class pairs.
#
# Return:
#      - List of all battery class pairs
#
# Created by: Gilbert
###################################
def SetChargeDischarge(data, battery_pairs):    
    all_batteries = []
    
    battery_pairs = battery_pairs[::-1]
    for b in battery_pairs:
        sell_period = b[0][::-1] # Reverse the order
        buy_period = b[1][::-1] # Reverse the order
        
        if len(sell_period)!= 0 or len(buy_period) != 0:
            sell_price = GetSpotPrice(data, sell_period)
            buy_price = GetSpotPrice(data, buy_period)

            battery = Battery(buy_period, buy_price, sell_period, sell_price)
            battery.NormalSetting()
            battery.Optimise()

            all_batteries.append(battery)
    
    return all_batteries

###################################
# A function to calculate daily revenue.
#
# Parameters:
#      - all_batteries : List of battery class pairs.
#
# Return:
#      - List of battery class pairs
#
# Created by: Gilbert
###################################
def ComputeDailyRevenue(all_batteries):
    revenues = 0
    for battery in all_batteries:
        revenues += battery.Revenue()
    return revenues

In [255]:
import time

# Test Period checkpoint 3
cp3_start_period = '2020-07-17 00:30:00'
cp3_end_period   = '2020-07-18 00:00:00'

vic_spot_price = market_data[['Time (UTC+10)', 'period', 'Regions VIC Trading Price ($/MWh)']]

cp_3 = vic_spot_price.loc[(vic_spot_price['Time (UTC+10)'] >= cp3_start_period) & \
        (vic_spot_price['Time (UTC+10)'] <= cp3_end_period)]

cp_3.to_csv('../../preprocessed_data/checkpoint3.csv', index = False)

check3 = pd.read_csv('../../preprocessed_data/checkpoint3.csv')

In [256]:
tic = time.perf_counter()
best_batteries = OrderedDict()
for s in range(4, 48):
    for b in range(5, 48):
        min_price_cp3, max_price_cp3 = GetMinMax(check3, buy_threshold = b, sell_threshold = s)
        battery_pairs = FindBatteryPairs(min_price_cp3, max_price_cp3)
        all_batteries = SetChargeDischarge(check3, battery_pairs)
        dailyrev = ComputeDailyRevenue(all_batteries)
        
        best_batteries[dailyrev] = all_batteries
            
toc = time.perf_counter()
best_revenue = max(best_batteries)
print(best_revenue)
print(f"Daily CV Duration is: {toc - tic:0.4f}")

battery = best_batteries[best_revenue]
raw_power = [0 for i in range(48)]
market_dispatch = [0 for i in range(48)]
opening_capacity = [0 for i in range(48)]
closing_capacity = [0 for i in range(48)]
for b in battery:
    for cp in range(len(b.charge_period)):
        raw_power[b.charge_period[cp][1] - 1] = b.charge_raw_power[cp]
        market_dispatch[b.charge_period[cp][1] - 1] = b.charge_market_dispatch[cp]
        opening_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][0]
        closing_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][1]
    
    for dp in range(len(b.discharge_period)):
        raw_power[b.discharge_period[dp][1] - 1] = b.discharge_raw_power[dp]
        market_dispatch[b.discharge_period[dp][1] - 1] = b.discharge_market_dispatch[dp]
        opening_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][0]
        closing_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][1]
    
    print("CHARGE")
    print("PERIOD : ", b.charge_period)
    print("PRICE : ", b.charge_price)
    print("RAW POWER : ", b.charge_raw_power)
    print("MARKET DISPATCH : ", b.charge_market_dispatch)
    print("CAPACITY : ", b.charge_capacity)
    print('\n')
    print("DISCHARGE")
    print("PERIOD : ", b.discharge_period)
    print("PRICE : ", b.discharge_price)
    print("RAW POWER : ", b.discharge_raw_power)
    print("MARKET DISPATCH : ", b.discharge_market_dispatch)
    print("CAPACITY : ", b.discharge_capacity)
    
    print("REVENUE : ", b.Revenue())
    print('\n')
    
check3['Raw Power'] = pd.Series(raw_power)
check3['Market Dispatch'] = pd.Series(market_dispatch)
check3['Opening Capacity'] = pd.Series(opening_capacity)
check3['Closing Capacity'] = pd.Series(closing_capacity)

49680.10698353515
Daily CV Duration is: 1.5100
CHARGE
PERIOD :  [(13, 7), (8, 8)]
PRICE :  [(7, 58.04), (8, 51.85)]
RAW POWER :  [-300, -300]
MARKET DISPATCH :  [-150.0, -150.0]
CAPACITY :  [[0, 135.0], [135.0, 270.0]]


DISCHARGE
PERIOD :  [[2, 17], [4, 21]]
PRICE :  [(17, 153.81), (21, 110.29)]
RAW POWER :  [300, 266.6666666666667]
MARKET DISPATCH :  [135.0, 120.00000000000001]
CAPACITY :  [[270.0, 120.0], [120.0, 0]]
REVENUE :  17059.958860898085


CHARGE
PERIOD :  [(16, 24)]
PRICE :  [(24, 63.2)]
RAW POWER :  [-300]
MARKET DISPATCH :  [-150.0]
CAPACITY :  [[0, 135.0]]


DISCHARGE
PERIOD :  [[23, 25]]
PRICE :  [(25, 71.71)]
RAW POWER :  [300]
MARKET DISPATCH :  [135.0]
CAPACITY :  [[135.0, 0]]
REVENUE :  27.62749631685074


CHARGE
PERIOD :  [(5, 27), (1, 28), (2, 29), (3, 30), (4, 31)]
PRICE :  [(27, 46.23), (28, 39.75), (29, 39.94), (30, 39.97), (31, 42.72)]
RAW POWER :  [-300, -300, -300, -300, -88.88888888888889]
MARKET DISPATCH :  [-150.0, -150.0, -150.0, -150.0, -44.44444444444

In [257]:
check3.to_csv('../../preprocessed_data/checkpoint3_optimised.csv', index = False)

In [ ]:
if len(sell_period) == len(sell_market_dispatch) and \
           len(buy_period) == len(buy_market_dispatch):
                
            battery.charge_market_dispatch.extend(buy_market_dispatch)
            battery.discharge_market_dispatch.extend(sell_market_dispatch)
        
        if len(sell_period) < len(buy_period):
            energy_required = 17
            battery.charge_market_dispatch.extend(buy_market_dispatch[:len(sell_period)])
            battery.charge_market_dispatch.append(energy_required)
            
            battery.discharge_market_dispatch.extend(sell_market_dispatch[:len(sell_period)])
        elif len(sell_period) == len(buy_period):
            battery.charge_market_dispatch.extend(buy_market_dispatch[:len(sell_period)])
            battery.discharge_market_dispatch.extend(sell_market_dispatch[:len(sell_period)])

In [8]:
minima = []
maxima = []

period = 48

start = 0
end = period 
while end <= len(market_data):
    data = market_data.iloc[start:end, :]
    price = data['Regions VIC Trading Price ($/MWh)']
    min_price, max_price = GetMinMax(price)
    tmp_minima, tmp_maxima = BuySellPeriod(min_price, max_price)
    
    minima.extend(tmp_minima)
    maxima.extend(tmp_maxima)
    start += period
    end += period

minima.append(float("nan"))
maxima.append(float("nan"))

market_data['minimum_price'] = pd.Series(minima)
market_data['maximum_price'] = pd.Series(maxima)

KeyError: 'Regions VIC Trading Price ($/MWh)'

In [ ]:
market_data